In [6]:
#| default_exp visuals

In [7]:
#| hide
from nbdev.showdoc import *

In [8]:
#| export
import numpy as np
import pandas as pd
import anndata as ad

In [9]:
from allos.readers_tests import *
from allos.preprocessing import subset_common_cells, transfer_obs


In [10]:
combined_mouse_data = process_mouse_data()


🔎 Looking for file at: /data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/allos/resources/e18.mouse.clusters.csv


FileNotFoundError: 
❌ File not found at: /data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/allos/resources/e18.mouse.clusters.csv

📂 Directory structure of /data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/allos/resources:


In [11]:
#| hide
from nbdev.showdoc import *

In [ ]:
import nbdev; nbdev.nbdev_export()